In [15]:
# -*- coding:utf8 -*-
import os
import csv
import pandas as pd
import time

path_train = "../data/dm/train.csv"  # 训练文件
path_test = "../data/dm/test.csv"  # 测试文件

path_test_out = "model/"  # 预测结果输出路径为model/xx.csv,有且只能有一个文件并且是CSV格式。


def read_csv():
    """
    文件读取模块，头文件见columns.
    :return: 
    """
    # for filename in os.listdir(path_train):
    tempdata = pd.read_csv(path_train)
    tempdata.columns = ["TERMINALNO", "TIME", "TRIP_ID", "LONGITUDE", "LATITUDE", "DIRECTION", "HEIGHT", "SPEED",
                        "CALLSTATE", "Y"]


def process():
    import pandas as pd
    import numpy as np
    
    """
    处理过程，在示例中，使用随机方法生成结果，并将结果文件存储到预测结果路径下。
    :return: 
    """
#build model:::
    print('building model..')
    from xgboost import XGBRegressor
    train_data = pd.read_csv(path_train)
   
    print('row/col of train :',train_data.shape)
    test_data = pd.read_csv(path_test)
  
    print('row/col of test  :',test_data.shape)
    input_label =[ "LONGITUDE", "LATITUDE", "DIRECTION", "HEIGHT", "SPEED","CALLSTATE"]
    output_label = ['Y']
#     train_input = train_data[input_label].values
#    train_output = train_data[output_label].values
#     test_input = test_data[input_label].values
    test_output = test_data[output_label].values
    
    #data-pre-process:
    train_input = []
    a = train_data.groupby(['TERMINALNO',"TRIP_ID"])
    c =list(a.groups)
    for tmp in c:
        b = a.get_group(tmp)
        if len(b)==1:
            continue
        for line in range(len(b)-1):
            index = [5,6,7,8,9]
            for k in index:
                if k < 8:
                    train_input.append(b.values[line+1][index] - b.values[line][index])
                else:
                    train_input.append(b.values[line][index])
            
    print('len of train :',len(train_input))                
                
    test_input = []
    a1 = test_data.groupby(['TERMINALNO',"TRIP_ID"])
    c1 =list(a1.groups)
    for tmp1 in c1:
        b1 = a1.get_group(tmp1)
        if len(b1)==1:
            continue
        for line in range(len(b1)-1):
            index = [5,6,7,8]
            for k in index:
                if k < 8:
                    test_input.append(b1.values[line+1][index] - b1.values[line][index])
                else:
                    test_input.append(b1.values[line][index])
    print('len of test :',len(test_input))
    
    train_input = pd.DataFrame(train_input)
    test_input =  pd.DataFrame(test_input)
    print('real row/col of train :',train_input.shape)
    print('real row/col of test  :',test_input.shape)
    print(train_input.head())
    reg = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=3, silent=True)
    reg.fit(train_input[[1,2,3]], train_input[4])
    y_pred = reg.predict(test_input[[1,2,3]])
    print(y_pred)
    
    mse = 0
    for ele in range(len(y_pred)):
        mse += (y_pred[ele] - test_output[ele][0])**2
    mse = mse / len(y_pred)
    print("mse : ", mse)
    
    with (open(os.path.join(path_test_out, "test.csv"), mode="w")) as outer:
        writer = csv.writer(outer)
        writer.writerow(["Id", "Pred"])
        mean = 0.0 
        k = 0
        sum = 0.0
        count = 0
        for i in range(len(test_input)-1):
            if(test_data.loc[i+1]["TERMINALNO"] == test_data.loc[i]["TERMINALNO"]):
                sum += y_pred[i]
                count += 1
            if(test_data.loc[i+1]["TERMINALNO"] != test_data.loc[i]["TERMINALNO"] or i == len(test_input)-2):
                mean = sum / (count+1)
                #print(mean,count)
                writer.writerow([test_data.loc[i]["TERMINALNO"], mean])
                k += 1
                count = 0
                sum = 0.0
                mean -= mean


if __name__ == "__main__":
    # 程序入口
    start = time.time()
    process()
    end = time.time()
    time_elapsed = end - start
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

building model..
row/col of train : (53284, 10)
row/col of test  : (16022, 10)
len of train : 241710
len of test : 58376
real row/col of train : (241710, 5)
real row/col of test  : (58376, 4)
      0          1     2    3    4
0  12.0  -0.091431  1.96  0.0  0.0
1  12.0  -0.091431  1.96  0.0  0.0
2  12.0  -0.091431  1.96  0.0  0.0
3  12.0  39.402588  2.15  0.0  0.0
4  12.0  39.402588  2.15  0.0  0.0
[0.4866439  0.4866439  0.4866439  ... 0.48518476 0.48518476 0.48646   ]


IndexError: index 16022 is out of bounds for axis 0 with size 16022

In [3]:
a = 1

b = a
a = 2
print(b)

1
